# NMR Assisted MS Metabolite Identification

Author: Olatomiwa Bifarin<br>
Department of Biochemistry and Molecular Biology<br>
University of Georgia<br>
Edison Lab<br>

Last edited: 21JAN2020

_This is a static version of a Jupyter notebook, and work (documentation) is still in progress_ 

In [40]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style
import scipy
#For Seaborn plots
import seaborn as sns; sns.set(style='white')
#To ignore warning
import warnings
warnings.filterwarnings('ignore')

# More sharp and legible graphics
%config InlineBackend.figure_format = 'retina'

# Set seaborn figure labels to 'talk', to be more visible. 
sns.set_context('talk', font_scale=0.8)

## 1. Import and Set up all (MS) features with < 0.05 _q_-values. 

_Import all 472 features_

In [41]:
qfeatures = pd.read_excel('RCC_472features.xlsx')
qfeatures.head()

,ID,Mode,RT [min],Name,Formula
0,1,positive,2.317,1,NaN
1,9,positive,1.901,5-Acetylamino-6-amino-3-methyluracil,C7 H10 N4 O3
2,94,positive,3.489,Pseudoephedrine,C10 H15 N O
3,95,positive,1.898,"3-(1H-1,2,4-Triazol-3-yl)alanine",C5 H8 N4 O2
4,147,positive,2.580,147,C33 H41 N O6 P2


_Import all MS features_

In [42]:
dfMS = pd.read_excel('data/RCC_S280_combinedData_7097.xlsx')
dfMS_trans = dfMS.T # Transpose dataframe
IDs = np.arange(1, dfMS_trans.shape[1] + 1) # Generate a range of numbers (7098) to replace nan values in the name row
IDs = pd.Series(IDs) # creating series 
dfMS_trans.iloc[4] = dfMS_trans.iloc[4].fillna(IDs) # replace nan with the IDs, and replace the name row

In [43]:
MS = dfMS_trans.drop(['Mode', 'Molecular Weight', 'Name', 'RT [min]', 'Formula'])
MS.index.name = 'MS_ID'
MS.reset_index(inplace=True) # Converts the row names into a column.
header = MS.iloc[0] # Create a new variable called 'header' from the first row of the dataset
MS = MS[1:] # Replace the dataframe with a new one which does not contain the first row
MS.rename(columns = header, inplace=True)# Rename the dataframe's column values with the header variable
MS.rename(columns={'ID':'Sample ID'}, inplace = True) # Rename column name
# To rest the row index to start from 0
MS.index = MS.index - 1 
MS.head()

,Sample ID,1,2,3,4,5,6,7,8,9,...,7088,7089,7090,7091,7092,7093,7094,7095,7096,7097
0,C001,7.24885e+08,1.10388e+09,5.06818e+06,4682.6,18261.9,1322.95,61663,191620,5.61943e+07,...,168447,81645.2,501574,43882.9,235674,141900,144189,1.65926e+06,105636,76801.7
1,C002,8.39383e+08,1.87456e+09,1.87657e+09,1.69931e+08,18267.8,1674.72,227698,532662,2.03319e+08,...,36254.4,242145,876199,274884,236921,167226,114217,771149,303623,37380.4
2,C003,7.65629e+08,1.67333e+09,7.47349e+06,8567.13,25730.9,3246.46,133905,495502,1.92514e+08,...,1.82785e+06,273594,946435,97021.7,839316,519959,467160,1.2216e+06,219606,6270.59
3,C004,1.25663e+08,1.1825e+09,1.17911e+09,6.72327e+06,10526.3,1076.99,129811,392377,1.16585e+08,...,450210,308199,343794,7427.46,275350,145776,64094.9,1.7718e+06,197519,60830.9
4,C005,1.06973e+07,8.79722e+08,8.56535e+08,438385,15913.8,1008.31,48496,122696,1.10997e+07,...,22167.5,39971.3,156251,718.724,199148,50598.8,107228,58713.6,71784.4,75589.5


_Import dataframe containing clinical ID_

In [44]:
path = "data/Sample_List_MS.xlsx"
fields = \
[
    'Sample ID',
    'Patient ID'
]

MSpatid = pd.read_excel(path)[fields]
MSpatid.head()

,Sample ID,Patient ID
0,C001,PD001
1,C002,PD003
2,C003,PD005
3,C004,PD012
4,C005,PD014


`MSData` is the dataframe for the MS Peak Intensities with <mark>Patient ID</mark> and <mark>Groups</mark>

In [45]:
MSData = pd.merge(MS, MSpatid, on='Sample ID') # Merge the two dataframe on Sample ID
MSData.head()

,Sample ID,1,2,3,4,5,6,7,8,9,...,7089,7090,7091,7092,7093,7094,7095,7096,7097,Patient ID
0,C001,7.24885e+08,1.10388e+09,5.06818e+06,4682.6,18261.9,1322.95,61663,191620,5.61943e+07,...,81645.2,501574,43882.9,235674,141900,144189,1.65926e+06,105636,76801.7,PD001
1,C002,8.39383e+08,1.87456e+09,1.87657e+09,1.69931e+08,18267.8,1674.72,227698,532662,2.03319e+08,...,242145,876199,274884,236921,167226,114217,771149,303623,37380.4,PD003
2,C003,7.65629e+08,1.67333e+09,7.47349e+06,8567.13,25730.9,3246.46,133905,495502,1.92514e+08,...,273594,946435,97021.7,839316,519959,467160,1.2216e+06,219606,6270.59,PD005
3,C004,1.25663e+08,1.1825e+09,1.17911e+09,6.72327e+06,10526.3,1076.99,129811,392377,1.16585e+08,...,308199,343794,7427.46,275350,145776,64094.9,1.7718e+06,197519,60830.9,PD012
4,C005,1.06973e+07,8.79722e+08,8.56535e+08,438385,15913.8,1008.31,48496,122696,1.10997e+07,...,39971.3,156251,718.724,199148,50598.8,107228,58713.6,71784.4,75589.5,PD014


In [46]:
lst = [qfeatures['ID'].tolist(), ['Patient ID']] # create a list of list. 
flat_list = [item for sublist in lst for item in sublist] # flatten out the list
len(flat_list)

473

In [47]:
MS_qfeatures = MSData.filter(flat_list)
MS_qfeatures.head()

,1,9,94,95,147,170,173,245,250,260,...,6939,6956,6972,6990,6996,6997,7001,7016,7087,Patient ID
0,7.24885e+08,5.61943e+07,8374.64,8.02358e+06,18762,1.72926e+07,7213.1,4.31192e+06,5.17764e+06,4.12823e+06,...,557204,55107.5,49981.4,5264.32,21831.6,4.24068e+06,310138,500365,36277.1,PD001
1,8.39383e+08,2.03319e+08,1.83963e+06,3.04003e+07,5047.83,2.07472e+07,4682.64,2.82463e+06,99855,2.72602e+06,...,6630.41,3611.89,27454.5,10926.7,142046,63006,209233,3368.27,280789,PD003
2,7.65629e+08,1.92514e+08,737077,3.08675e+07,17108.2,1.87394e+07,16723.3,2.8226e+06,3.11985e+06,2.69634e+06,...,1.90054e+06,528003,58124.6,6916.87,228388,774392,261771,183970,274158,PD005
3,1.25663e+08,1.16585e+08,17730.4,1.67902e+07,3953.93,2.64819e+06,3551.59,2.50321e+06,37816.2,2.38551e+06,...,18563.6,19629.2,22933.2,47554.2,178849,16533.8,2338.97,7458.13,122430,PD012
4,1.06973e+07,1.10997e+07,6598.17,1.65861e+06,2775.03,85782.6,3561.69,4.476e+06,122586,4.24929e+06,...,1103.74,10201.6,93332.9,589.207,5127.06,7152.02,957.374,235492,17133.6,PD014


In [48]:
MS_qfeatures.shape

(280, 473)

## 2. Import and Setup all NMR features. 

In [49]:
dfNMR = pd.read_excel('data/binned_NMRfeatures_22JAN2020.xlsx')
dfNMR.drop(['Run_ID', 'Sample_description', 
            'Sample_grp', 'Yvec', 'Run_IDOrig'], axis=1, inplace=True)
dfNMR.head()

,Sample_ID,unk1,unk2,bile_acid1,bile_acid2,HIVA,lactate,unk3,acetate,acetone,...,hypoxanthine_1,hypoxanthine_2,formate,unk10,Trigonelline_1,Trigonellinamide_1,Trigonellinamide_2,Trigonelline_2,Trigonellinamide_3,unk11
0,2051,0.001712,0.009648,0.004468,0.003858,0.081140,0.14363,0.12591,0.12119,0.106510,...,0.010820,0.013258,0.022455,0.025070,0.172890,0.008710,0.008855,0.085666,0.006842,0.000836
1,2063,0.002893,0.005114,0.004316,0.003724,0.097917,0.18802,0.11883,0.12937,0.101940,...,0.017549,0.015019,0.012625,0.000710,0.033725,0.009957,0.009227,0.019867,0.010659,0.000375
2,2140,0.001662,0.004722,0.003204,0.002227,0.091223,0.21726,0.15563,0.10034,0.099316,...,0.013705,0.012282,0.015653,0.021196,0.107950,0.005262,0.004900,0.077770,0.004254,0.002216
3,2043,0.002624,0.004493,0.003701,0.003296,0.101520,0.17155,0.14087,0.10723,0.108910,...,0.010088,0.016178,0.044723,0.007703,0.041618,0.010619,0.009132,0.021388,0.011091,0.000675
4,2164,0.003097,0.002801,0.004016,0.002949,0.108540,0.15723,0.15420,0.11200,0.137320,...,0.010246,0.011751,0.016592,0.001621,0.028284,0.008392,0.007624,0.013508,0.008742,-0.000465


In [50]:
dfNMR.rename(columns={'Sample_ID':'Patient ID'}, inplace = True) # Rename column name
dfNMR.head()

,Patient ID,unk1,unk2,bile_acid1,bile_acid2,HIVA,lactate,unk3,acetate,acetone,...,hypoxanthine_1,hypoxanthine_2,formate,unk10,Trigonelline_1,Trigonellinamide_1,Trigonellinamide_2,Trigonelline_2,Trigonellinamide_3,unk11
0,2051,0.001712,0.009648,0.004468,0.003858,0.081140,0.14363,0.12591,0.12119,0.106510,...,0.010820,0.013258,0.022455,0.025070,0.172890,0.008710,0.008855,0.085666,0.006842,0.000836
1,2063,0.002893,0.005114,0.004316,0.003724,0.097917,0.18802,0.11883,0.12937,0.101940,...,0.017549,0.015019,0.012625,0.000710,0.033725,0.009957,0.009227,0.019867,0.010659,0.000375
2,2140,0.001662,0.004722,0.003204,0.002227,0.091223,0.21726,0.15563,0.10034,0.099316,...,0.013705,0.012282,0.015653,0.021196,0.107950,0.005262,0.004900,0.077770,0.004254,0.002216
3,2043,0.002624,0.004493,0.003701,0.003296,0.101520,0.17155,0.14087,0.10723,0.108910,...,0.010088,0.016178,0.044723,0.007703,0.041618,0.010619,0.009132,0.021388,0.011091,0.000675
4,2164,0.003097,0.002801,0.004016,0.002949,0.108540,0.15723,0.15420,0.11200,0.137320,...,0.010246,0.011751,0.016592,0.001621,0.028284,0.008392,0.007624,0.013508,0.008742,-0.000465


## 3. Select Samples Common to both MS and NMR

In [51]:
print(dfNMR.shape)
print(MS_qfeatures.shape)

(284, 49)
(280, 473)


_Merge NMR and MS dataframes  on Patient ID_

In [52]:
combined = pd.merge(MS_qfeatures, dfNMR, on='Patient ID') # Merge the two dataframe on Sample ID
combined.shape

(256, 521)

_Select Samples Common to both MS and NMR_

In [53]:
final_NMR = dfNMR[dfNMR['Patient ID'].isin(combined['Patient ID'])]
final_MS = MS_qfeatures[MS_qfeatures['Patient ID'].isin(combined['Patient ID'])]
print(final_NMR.shape)
print(final_MS.shape)

(256, 49)
(256, 473)


## 4. Correlations

In [54]:
final_NMR.drop(['Patient ID'], axis=1, inplace=True)
final_MS.drop(['Patient ID'], axis=1, inplace=True)

In [55]:
NMRcorr_compd, MScorr_compd = [], []
for column in final_NMR.columns:
    for column2 in final_MS.columns:
        col_corr = pd.DataFrame(np.corrcoef(final_NMR[column].astype(float), 
                                            final_MS[column2].astype(float))).abs()
        # Select upper triangle of correlation matrix
        #corr = col_corr.where(np.triu(np.ones(col_corr.shape), k=1).astype(np.bool))
        if col_corr[1][0] > 0.30: 
            #column to drop NMR's; column2 to drop MS's 
            NMRcorr_compd.append(column)
            MScorr_compd.append(column2)

In [72]:
NMRcorr_compd;

In [91]:
MScorr_compd;

In [92]:
corr_result = pd.DataFrame(
    {'NMR Features': NMRcorr_compd,
     'MS Features': MScorr_compd
    })
corr_result

,NMR Features,MS Features
0,lactate,5065
1,lactate,5216
2,lactate,5379
3,lactate,5393
4,lactate,7001
5,acetate,94
6,acetate,3035
7,acetate,3632
8,acetone,6885
9,acetoacetate,2577


In [93]:
# Import MS_labels
MS_labels = pd.read_excel('data/MS_labels.xlsx')
# Check for potential MS ID.
MSlabels_corr = MS_labels[MS_labels.ID.isin(MScorr_compd)]
MSlabels_corr

,ID,Mode,RT [min],Name,Formula
93,94,positive,3.489,Pseudoephedrine,C10 H15 N O
1690,1691,positive,3.418,1691,NaN
1770,1771,positive,3.001,1771,C13 H24 N4 O4
1808,1809,positive,0.910,1809,C8 H20 N5 O5 P
2030,2031,positive,0.900,2031,C10 H24 N2 O4 P2
2159,2160,positive,0.898,Combretastatin A-4,C18 H20 O5
2576,2577,positive,1.457,D-Pipecolicacid,C6 H11 N O2
2787,2788,positive,2.615,6-Methyl-2H-chromen-2-one,C10 H8 O2
3034,3035,positive,4.530,3035,C31 H10 N3 O18 P
3126,3127,positive,1.232,3127,C6 H6 N10 O
